Passo 3 - Gerar a visão geral das modularidades identificadas após os calculos feitos no Gephi

Notebook responsável criar a visão geral da análise de modularidades encontradas na extração dos tweets feitos no passo 1, a partir dos dados calculados pelo Gephi 

Declarar as libs

In [297]:
import pandas as pd
import dataframe_image as dfi

from wordcloud import WordCloud
from wordcloud import ImageColorGenerator
from wordcloud import STOPWORDS
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
import networkx as nx
import numpy as np
import re
import os
from nltk.probability import FreqDist

from collections import Counter
from operator import itemgetter
import tweepy

pd.set_option('display.max_rows', 1000)

Inserir o token de autenticação do Twitter 

In [298]:
client = tweepy.Client(bearer_token='Insira o token de acesso', wait_on_rate_limit=True)


Definição dos arquivos de entrada do processamento

In [ ]:
rotulo = "Nome do Arquivo" # nome do arquivo que será usado como padrão para criação dos arquivos
data_pesquisa = "0101" # data referencia usada para criação dos arquivos
save_path = 'C:\\diretório\\' # diretório padrão para criação dos arquivos
arq_gephi = save_path + "\\" + rotulo + "\\" + 'gephi_' + rotulo + '.csv'
arq_fonte = save_path + "\\" + rotulo + "\\" + rotulo + '_CR_'+ data_pesquisa + '_raw.csv'
arq_contas_brasil = 'twitter_contas_br_2022_v3.csv'



Leitura dos arquivos de entrada

In [305]:
df_c = pd.read_csv(arq_contas_brasil, sep=';', low_memory = False, index_col=False)
df_a = pd.read_csv(arq_gephi, low_memory = False, index_col=False)
df_b = pd.read_csv(arq_fonte, low_memory = False, index_col=False)

df_a_trab = pd.DataFrame(df_a, columns = ["Id", "modularity_class"]) 

df_a_trab['Id_convert'] = df_a_trab['Id'].astype(str)

df_b_trab = pd.DataFrame(df_b, columns = ["author_id", "text"]) 

df_b_trab['Id_convert'] = df_b_trab['author_id'].astype(str)

df_b_trab.rename(columns = {'author_id':'Id'}, inplace = True)

df_merge_inner = pd.merge(df_a_trab, df_b_trab, how = 'inner', on = 'Id_convert')

df_merge_inner.shape

Preparação dos dados para gerar Nuvem de Palavras

In [307]:

Lista_palavras = ["vai","agora","HTTPS","Se", "O", "ao","de","a","o","que","e","do","da","em","um","para","é","com","não","uma","os","no","se","na","por","mais","as","dos","como","mas","foi","ao","ele","das","tem","à","seu","sua","ou","ser","quando","muito","há","nos","já","está","eu","também","só","pelo","pela","até","isso","ela","entre","era","depois","sem","mesmo","aos","ter","seus","quem","nas","me","esse","eles","estão","você","tinha","foram","essa","num","nem","suas","meu","às","minha","têm","numa","pelos","elas","havia","seja","qual","será","nós","tenho","lhe","deles","essas","esses","pelas","este","fosse","dele","tu","te","vocês","vos","lhes","meus","minhas","teu","tua","teus","tuas","nosso","nossa","nossos","nossas","dela","delas","esta","estes","estas","aquele","aquela","aqueles","aquelas","isto","aquilo","estou","está","estamos","estão","estive","esteve","estivemos","estiveram","estava","estávamos","estavam","estivera","estivéramos","esteja","estejamos","estejam","estivesse","estivéssemos","estivessem","estiver","estivermos","estiverem","hei","há","havemos","hão","houve","houvemos","houveram","houvera","houvéramos","haja","hajamos","hajam","houvesse","houvéssemos","houvessem","houver","houvermos","houverem","houverei","houverá","houveremos","houverão","houveria","houveríamos","houveriam","sou","somos","são","era","éramos","eram","fui","foi","fomos","foram","fora","fôramos","seja","sejamos","sejam","fosse","fôssemos","fossem","for","formos","forem","serei","será","seremos","serão","seria","seríamos","seriam","tenho","tem","temos","tém","tinha","tínhamos","tinham","tive","teve","tivemos","tiveram","tivera","tivéramos","tenha","tenhamos","tenham","tivesse","tivéssemos","tivesse","tiver","tivermos","tiverem","terei","terá","teremos","terão","teria","teríamos","teriam","Se","RT","se","ao", "outro", "outra'"]

text = " ".join(i for i in df_merge_inner.text)

stop_words = Lista_palavras + list(STOPWORDS)
regex_pattern = re.compile(pattern = "["
        u"\U0001F600-\U0001F64F" 
        u"\U0001F300-\U0001F5FF" 
        u"\U0001F680-\U0001F6FF" 
        u"\U0001F1E0-\U0001F1FF"  
                           "]+", flags = re.UNICODE)

text_limpo_a = re.sub(regex_pattern,'',text)

re_list = ['@[A-Za-z0–9_]+', '#']
combined_re = re.compile( '|'.join( re_list) )
text_limpo_b = re.sub(combined_re,'',text_limpo_a)


Gerar Nuvem de Palavras

In [ ]:
wordcloud = WordCloud(max_words=100, collocation_threshold = 10, width = 1500, height = 1000, random_state = 42, collocations = False, colormap = 'tab10', stopwords = stop_words, background_color="white").generate(text_limpo_b)

plt.style.use('ggplot')
plt.figure( figsize=(15,10))
plt.title("Wordcloud - Dados Gerais / ")
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.tight_layout()

plt.savefig(save_path + "\\" + rotulo + "\\" + rotulo + "_wordcloud_geral.png", format='png')

plt.show()

Identificar frenquencia das palavras

In [ ]:
frequencia_palavras = FreqDist(text_limpo_b)
frequencia_palavras

Preparação dos dados para gerar as visões de centralidade de entrada (Inbound)

In [ ]:
df_a_centralidade_inbound = df_a.query('indegree > 0 ')[['Id','eigencentrality', 'pageranks', 'Authority', 'weighted indegree', 'modularity_class']]
df_a_centralidade_inbound = df_a_centralidade_inbound.sort_values(['weighted indegree'], ascending=[False])
df_a_centralidade_inbound

Gráfico de interpolação das centralidades de entrada

In [ ]:
z = df_a_centralidade_inbound['eigencentrality']
x = df_a_centralidade_inbound['pageranks']
y = df_a_centralidade_inbound['Authority']
 
fig = plt.figure(figsize = (10, 8))
ax = plt.axes(projection ="3d")
   
ax.grid( color ='grey', 
        linestyle ='-.', linewidth = 0.3, 
        alpha = 0.2) 
 
 
my_cmap = plt.get_cmap('hsv')
 
sctt = ax.scatter3D(x, y, z,
                    alpha = 0.8,
                    c = (x + y + z), 
                    s =100,
                    cmap = my_cmap, 
                    marker ='o')
 
plt.title("Centralidades aplicadas aos Nodes - Modularidade")
ax.set_xlabel('pageranks', fontweight ='bold') 
ax.set_ylabel('Authority', fontweight ='bold') 
ax.set_zlabel('eigencentrality', fontweight ='bold')
plt.margins(0.2)
plt.tight_layout()
 
    
plt.savefig(save_path + "\\" + rotulo + "\\" + rotulo + "_centralidade_geral.png", format='png')
    
    
plt.show()

Geração dos arquivos de centralidade de entrada

In [312]:
df_c['Id']=df_c['Id'].str.lower()
df_a_centralidade_inbound['Id']=df_a_centralidade_inbound['Id'].str.lower()

df_a_centralidade_inbound_classificado = df_a_centralidade_inbound.merge(df_c, on='Id', how='left')
df_a_centralidade_inbound_classificado.head(11)

csv_centralidade_geral = save_path + "\\" + rotulo + "\\" + rotulo + "_Tabela Centralidade Geral.csv"

df_a_centralidade_inbound_classificado.head(11).to_csv(csv_centralidade_geral)

Preparação dos dados para gerar as visões de centralidade de saída (Outbound)

In [ ]:
df_a_centralidade_outbound = df_a.query('outdegree > 0 ')[['Id', 'weighted outdegree', 'modularity_class']]
df_a_centralidade_outbound = df_a_centralidade_outbound.sort_values(['weighted outdegree'], ascending=[False])
df_c['Id']=df_c['Id'].str.lower()
df_a_centralidade_outbound['Id']=df_a_centralidade_outbound['Id'].str.lower()

Geração dos arquivos de centralidade de saída - usuários

In [317]:
df_a_centralidade_outbound_classificado = df_a_centralidade_outbound.merge(df_c, on='Id', how='left')
df_a_centralidade_outbound_classificado.head(11)

csv_Top10_senders = save_path + "\\" + rotulo + "\\" + rotulo + "_Tabela TOP10 Outbound Users.csv"
df_a_centralidade_outbound_classificado.head(11).to_csv(csv_Top10_senders)

Geração dos arquivos de centralidade de saída - modularidades

In [ ]:
grupos_outbound = df_a_centralidade_outbound_classificado.groupby(['modularity_class']).sum()
grupos_outbound = grupos_outbound.sort_values('weighted outdegree', ascending=False)
grupos_outbound['percent_total'] = (grupos_outbound['weighted outdegree'] / grupos_outbound['weighted outdegree'].sum())*100
grupos_outbound['weighted outdegree'].sum()
grupos_outbound.head(11)
csv_top10_comunidades = save_path + "\\" + rotulo + "\\" + rotulo + "_Tabela TOP10 Outbound Comunidades.csv"
grupos_outbound.head(11).to_csv(csv_top10_comunidades)

Geração de um gráfico de pizza para melhor visualização da distribuição das modularidades 

In [ ]:
data_grafico = grupos_outbound.head(11)
sizes = data_grafico['percent_total']

m_explode = (0, 0.1, 0, 0)  

fig1, ax1 = plt.subplots()
ax1.pie(sizes, autopct='%1.1f%%', 
        shadow=True, startangle=90)
ax1.axis('equal') 

plt.savefig(save_path + "\\" + rotulo + "\\" + rotulo + "_grafico_pie_comunidades.png", format='png')


plt.show()